# Install all required Libraries

In [1]:
conda install -c anaconda beautifulsoup4

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    numpy-base-1.15.4          |   py36h81de0dd_0         4.2 MB  anaconda
    numpy-1.15.4               |   py36h1d66e8a_0          35 KB  anaconda
    beautifulsoup4-4.8.1       |           py36_0         153 KB  anaconda
    openssl-1.1.1              |       h7b6447c_0         5.0 MB  anaconda
    soupsieve-1.9.5            |           py36_0          61 KB  anaconda
    mkl_fft-1.0.6              |   py36h7dd41cf_0         150 KB  anaconda
    certifi-2019.11.28         |           py36_0         156 KB  a

In [2]:
pip install lxml

     |████████████████████████████████| 5.8MB 14.2MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
!conda install -c conda-forge geopy --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         240 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0         conda-forge
    geopy:         1.20.0-py_0       conda-forge

The following p

In [4]:
import numpy as np 
import pandas as pd 

import json
import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

from bs4 import BeautifulSoup

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
from geopy.geocoders import Nominatim

# Extract the 'List of US Cities by population' content by doing web scarping of the wiki page.

In [6]:
link = 'https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population'
page = requests.get(link) 
soup = BeautifulSoup(page.text)
table = soup.find_all('table')[4]

## Convert web page data to data frame by defining the column name.

In [7]:
table_rows = table.find_all('tr')
res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)
df = pd.DataFrame(res, columns=["Rank", "City", "State", "estimated population", "enumerated population", "percentage change", "Sq.Area in Miles", "Sq.Area in KM", "population density in Sq Mi", "Population density in Km2", "Location"])
df.head()

,Rank,City,State,estimated population,enumerated population,percentage change,Sq.Area in Miles,Sq.Area in KM,population density in Sq Mi,Population density in Km2,Location
0,1,New York[d],New York,"8,398,748","8,175,133",+2.74%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W﻿...
1,2,Los Angeles,California,"3,990,456","3,792,621",+5.22%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°...
2,3,Chicago,Illinois,"2,705,994","2,695,598",+0.39%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W﻿...
3,4,Houston[3],Texas,"2,325,502","2,100,263",+10.72%,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W﻿...
4,5,Phoenix,Arizona,"1,660,272","1,445,632",+14.85%,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°...


### Lets find the radius of each city with the help of Sq.Area.

In [8]:
new= df["Sq.Area in Miles"].str.split("s", n=1, expand = True)
new = new[0].str.replace(u'\xa0',u'')
df["Sq.Area in Miles"] = new.str.replace(',','')
df["Sq.Area in Miles"] = df["Sq.Area in Miles"].astype(float)
df["Radius"] = np.sqrt(df["Sq.Area in Miles"])


df["Location"]= df["Location"].str.split("/", n = 2, expand = True)[1]
df.head()

,Rank,City,State,estimated population,enumerated population,percentage change,Sq.Area in Miles,Sq.Area in KM,population density in Sq Mi,Population density in Km2,Location,Radius
0,1,New York[d],New York,"8,398,748","8,175,133",+2.74%,301.5,780.9 km2,"28,317/sq mi","10,933/km2",﻿40.6635°N 73.9387°W﻿,17.363755
1,2,Los Angeles,California,"3,990,456","3,792,621",+5.22%,468.7,"1,213.9 km2","8,484/sq mi","3,276/km2",﻿34.0194°N 118.4108°W﻿,21.649480
2,3,Chicago,Illinois,"2,705,994","2,695,598",+0.39%,227.3,588.7 km2,"11,900/sq mi","4,600/km2",﻿41.8376°N 87.6818°W﻿,15.076472
3,4,Houston[3],Texas,"2,325,502","2,100,263",+10.72%,637.5,"1,651.1 km2","3,613/sq mi","1,395/km2",﻿29.7866°N 95.3909°W﻿,25.248762
4,5,Phoenix,Arizona,"1,660,272","1,445,632",+14.85%,517.6,"1,340.6 km2","3,120/sq mi","1,200/km2",﻿33.5722°N 112.0901°W﻿,22.750824


### Clean the data by droping the unnecessary columns such as Rank, estimated population, ... Sq.Area in Miles", "Sq.Area in KM, population density in Sq Mi.

In [9]:
df.drop(columns = ["Rank", "estimated population", "enumerated population", "percentage change", "Sq.Area in Miles", "Sq.Area in KM", "population density in Sq Mi"], inplace = True)
df.head()

,City,State,Population density in Km2,Location,Radius
0,New York[d],New York,"10,933/km2",﻿40.6635°N 73.9387°W﻿,17.363755
1,Los Angeles,California,"3,276/km2",﻿34.0194°N 118.4108°W﻿,21.649480
2,Chicago,Illinois,"4,600/km2",﻿41.8376°N 87.6818°W﻿,15.076472
3,Houston[3],Texas,"1,395/km2",﻿29.7866°N 95.3909°W﻿,25.248762
4,Phoenix,Arizona,"1,200/km2",﻿33.5722°N 112.0901°W﻿,22.750824


In [10]:
new = df["Location"].str.split(" ", n = 0, expand = False)
k = df.copy(deep = True)
k.head()

,City,State,Population density in Km2,Location,Radius
0,New York[d],New York,"10,933/km2",﻿40.6635°N 73.9387°W﻿,17.363755
1,Los Angeles,California,"3,276/km2",﻿34.0194°N 118.4108°W﻿,21.649480
2,Chicago,Illinois,"4,600/km2",﻿41.8376°N 87.6818°W﻿,15.076472
3,Houston[3],Texas,"1,395/km2",﻿29.7866°N 95.3909°W﻿,25.248762
4,Phoenix,Arizona,"1,200/km2",﻿33.5722°N 112.0901°W﻿,22.750824


In [11]:
Latitude = []
Longitude = []
for i in range(len(new)):
    Latitude.append(new[i][1][:-2])
    Longitude.append(new[i][2][:-3]) 

k["Latitude"] = Latitude
k["Longitude"] = Longitude
k["Latitude"] = k["Latitude"].str.replace(u'\ufeff',u'')
k.drop(columns = ["Location"], inplace = True)
k.head()
df = k.copy(deep = True)

In [12]:
df['Longitude'] = -df['Longitude'].astype(float)
df['Latitude'] = df['Latitude'].astype(float)
df['Radius'] = df['Radius']* 1000
df.head()

,City,State,Population density in Km2,Radius,Latitude,Longitude
0,New York[d],New York,"10,933/km2",17363.755354,40.6635,-73.9387
1,Los Angeles,California,"3,276/km2",21649.480363,34.0194,-118.4108
2,Chicago,Illinois,"4,600/km2",15076.471736,41.8376,-87.6818
3,Houston[3],Texas,"1,395/km2",25248.762346,29.7866,-95.3909
4,Phoenix,Arizona,"1,200/km2",22750.824161,33.5722,-112.0901


# Fetch the Per Capita Income State Wise for USA same as done for Population Density

In [13]:
wiki_url = "https://en.wikipedia.org/wiki/List_of_United_States_counties_by_per_capita_income"
source = requests.get(wiki_url).text
soup = BeautifulSoup(source, 'lxml')
neigh_table=soup.find_all('table')[2]

neigh_table

column_names = ['Rank', 'Country-equivalent', 'State', 'Per capita income', 'del2', 'del3', 'Population', 'del5']
df_wiki = pd.DataFrame(columns = column_names)

for tr_cell in neigh_table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
        #print(row_data)
    if len(row_data)==8:
        df_wiki.loc[len(df_wiki)] = row_data

df_wiki.head()

,Rank,Country-equivalent,State,Per capita income,del2,del3,Population,del5
0,1,New York County,New York,"$62,498","$69,659","$84,627","1,605,272","736,192"
1,2,Arlington,Virginia,"$62,018","$103,208","$139,244","214,861","94,454"
2,3,Falls Church City,Virginia,"$59,088","$120,000","$152,857","12,731","5,020"
3,4,Marin,California,"$56,791","$90,839","$117,357","254,643","102,912"
4,5,Alexandria City,Virginia,"$54,608","$85,706","$107,511","143,684","65,369"


## Cleaning the data by dropping unnecessary columns such as Rank, del2, del3, del5, the same way we did for population density.

In [14]:
df_wiki.drop(columns = ['Rank','del2', 'del3', 'del5'], axis = 1, inplace = True)

In [15]:
df_wiki.head()

,Country-equivalent,State,Per capita income,Population
0,New York County,New York,"$62,498","1,605,272"
1,Arlington,Virginia,"$62,018","214,861"
2,Falls Church City,Virginia,"$59,088","12,731"
3,Marin,California,"$56,791","254,643"
4,Alexandria City,Virginia,"$54,608","143,684"


## Plotting all the cities of USA to visualize it on Map.

In [16]:
!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [17]:
# create map of USA cities that we have using latitude and longitude values

map_tohood = folium.Map(location=[37.0902,-95.7129], tiles='Stamen Toner', zoom_start=5)

# add markers to map
for lat, lng, state, city in zip(df['Latitude'], df['Longitude'], df['State'], df['City']):
    label = '{}, {}'.format(city, state)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#7844cc',
        fill_opacity=0.6,
        parse_html=False).add_to(map_tohood)  
    
map_tohood

### We need to choose the location from the cities and so we need to get the venues details. We can get the venue details from Four Square Venue API.

In [21]:
CLIENT_ID = '4AUW3UKAAYQ31AONWDSDAH2GBDCVLAJE4TJFGE3P1VEI0ZTC'
CLIENT_SECRET = 'SH41RX42WBOFOLN2JDCPWELSSXZHVM2R13BVJRW5XGTIGT5B' 
VERSION = '20180323'
LIMIT=20

#### Function that extracts necessary columns into a data frame from the json files that we get when we search using four square API out of the values that we have in the data frame.

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius):
    
    venues_list=[]
    for name, lat, lng,radius in zip(names, latitudes, longitudes,radius):
       # print ('started')    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        #print (results)
        #print ('REQUESTED') 
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
       # print ('listed') 

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Category']
    
    return(nearby_venues)

In [27]:
df_venues = getNearbyVenues(names = df['City'], latitudes = df['Latitude'],longitudes = df['Longitude'], radius = df['Radius'])
df_ven = df_venues.copy()
df_venues.head()

,City,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Category
0,New York[d],40.6635,-73.9387,Brooklyn Botanic Garden,40.667622,-73.963191,Botanical Garden
1,New York[d],40.6635,-73.9387,Covenhoven,40.675143,-73.960203,Beer Bar
2,New York[d],40.6635,-73.9387,Brooklyn Museum,40.671521,-73.963677,Art Museum
3,New York[d],40.6635,-73.9387,Kings Theatre,40.646110,-73.957175,Theater
4,New York[d],40.6635,-73.9387,Prospect Park (Nethermead),40.660717,-73.968587,Field


In [28]:
df_cat = df_venues.Category.unique()
df_cat

array(['Botanical Garden', 'Beer Bar', 'Art Museum', 'Theater', 'Field',
       'Park', 'Bar', 'Farmers Market', 'Ice Cream Shop', 'Gourmet Shop',
       'Donut Shop', 'Juice Bar', 'Bookstore', 'Italian Restaurant',
       'Indie Movie Theater', 'Wine Shop', 'Climbing Gym',
       'Salon / Barbershop', 'Coffee Shop', 'Pizza Place', 'Trail',
       'Scenic Lookout', 'Gymnastics Gym', 'Grocery Store', 'Art Gallery',
       'Chinese Restaurant', 'Film Studio', 'New American Restaurant',
       'Harbor / Marina', 'Brewery', 'Mexican Restaurant', 'Rock Club',
       'Liquor Store', 'Sandwich Place', 'Stadium', 'Yoga Studio',
       'African Restaurant', 'Hotel', 'Café', 'Flower Shop', 'Beer Store',
       'Dessert Shop', 'American Restaurant', 'BBQ Joint', 'Salad Place',
       'Beer Garden', 'Concert Hall', 'Cheese Shop', 'Taco Place',
       "Men's Store", 'Seafood Restaurant', 'Convenience Store', 'Market',
       'Burger Joint', 'Record Shop', 'Spa', 'Wine Bar',
       'Greek Restaurant

## Lets assign weights to some of the categories that are prioritize by the CEO of the Company.

In [29]:
k = df_venues.copy(deep = True)
weights_dict={'Financial or Legal Service':5,'Market':5,'Business Service':5,'Office':5,'Building':5,'Shopping Mall':4,'Electronics Store':4,'Hardware Store':4,'Warehouse Store':3,'Community Center':3}
data = df_venues['Category']
allVenues = data.values.tolist()
#allVenues.Category.unique()

In [30]:
weights = []
for i in allVenues:
    if i in weights_dict.keys():
        weights.append(weights_dict[i])
    else :
        weights.append(0)
df_venues['weights'] = weights;
df_venues.head()

,City,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Category,weights
0,New York[d],40.6635,-73.9387,Brooklyn Botanic Garden,40.667622,-73.963191,Botanical Garden,0
1,New York[d],40.6635,-73.9387,Covenhoven,40.675143,-73.960203,Beer Bar,0
2,New York[d],40.6635,-73.9387,Brooklyn Museum,40.671521,-73.963677,Art Museum,0
3,New York[d],40.6635,-73.9387,Kings Theatre,40.646110,-73.957175,Theater,0
4,New York[d],40.6635,-73.9387,Prospect Park (Nethermead),40.660717,-73.968587,Field,0


In [31]:
# Dropping the rows that we are not giving any weight
df_venues.drop(df_venues[df_venues.weights < 1.0].index, inplace=True)
df_venues.head()

,City,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Category,weights
85,Phoenix,33.5722,-112.0901,Luci's at The Orchard,33.540604,-112.056737,Market,5
88,Phoenix,33.5722,-112.0901,Uptownfarmers Market,33.523100,-112.071516,Market,5
115,Philadelphia[e],40.0094,-75.1333,Czerw's Kielbasa,39.986034,-75.097974,Market,5
271,Columbus,39.9852,-82.9848,North Market,39.971952,-83.004338,Market,5
353,Denver[i],39.7619,-104.8811,City Floral,39.739205,-104.917126,Hardware Store,4


### Here we don't need to copy any unnecessary columns and only the relevant columns like city and weights are required. We need to group all the venues by city and calculating the means for each city.

In [32]:
citywise_venues_weights = df_venues[['City','weights']].copy()
citywise_venues_weights_means = citywise_venues_weights.groupby(['City']).mean()
citywise_venues_weights_means = citywise_venues_weights_means.reset_index(drop=False)
citywise_venues_weights_means.head()

,City,weights
0,Anaheim,3.0
1,Arlington,3.5
2,Augusta[t],4.0
3,Aurora,4.0
4,Bakersfield,4.0


#### Merging the table for which we calculated the means of weights city wise to the table.

In [33]:
city_selection = pd.merge(df, citywise_venues_weights_means, on='City')
city_selection = city_selection[['City','Population density in Km2','weights']].copy()
city_selection.head()

,City,Population density in Km2,weights
0,Phoenix,"1,200/km2",5.0
1,Philadelphia[e],"4,511/km2",5.0
2,Columbus,"1,520/km2",5.0
3,Denver[i],"1,746/km2",4.0
4,Washington[j],"4,304/km2",4.0


In [34]:
# Preprocessing the population density in Km2 column as we have to normalize these values
k = city_selection.copy(deep = True)
k['Population density in Km2'] = k['Population density in Km2'].str.split("/", n = 0, expand = True)
k['Population density in Km2'] = k['Population density in Km2'].str.replace(',','')
k['Population density in Km2'] = k['Population density in Km2'].astype(float)
city_selection = k.copy(deep = True)
city_selection.head()

,City,Population density in Km2,weights
0,Phoenix,1200.0,5.0
1,Philadelphia[e],4511.0,5.0
2,Columbus,1520.0,5.0
3,Denver[i],1746.0,4.0
4,Washington[j],4304.0,4.0


#### Normalizing our data frame

In [35]:
# Normalizing the data frame
from sklearn import preprocessing
column_names_to_normalize = ['Population density in Km2', 'weights']
x = city_selection[column_names_to_normalize].values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
city_selection[column_names_to_normalize] = pd.DataFrame(x_scaled)
city_selection.head()

,City,Population density in Km2,weights
0,Phoenix,0.208252,1.0
1,Philadelphia[e],0.923526,1.0
2,Columbus,0.277382,1.0
3,Denver[i],0.326204,0.5
4,Washington[j],0.878808,0.5


### Let's calculate the sum of normalized columns to determine the city that has maximum sum and conclude that one locality in that city would be the best fit 

In [36]:
#calculating the sum of normalized columns to determine the city that has maximum sum and conclude that one locality in that city would be the best fit 
city_selection['sum'] = city_selection['Population density in Km2'] + city_selection['weights']
row_num = city_selection['sum'].argmax()
city_name = city_selection['City'].iloc[row_num]
city_name


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  This is separate from the ipykernel package so we can avoid doing imports until


'Santa Ana'

In [37]:
# Finding the state in which that city belongs
row = df.loc[df['City']== city_name].index[0]
state_name = df['State'].iloc[row]
state_name

'California'

#### The Company CEO thinks that the state in which he will setup the new marketing office should have a Per Capita Income of at least of 50,000 USD, let us check if it satisfy the same.

In [38]:
# checking the percapita income of California 
p_row = df_wiki.loc[df_wiki['State'] == state_name].index[0]
per_capital_income = df_wiki['Per capita income'].iloc[p_row]
print("Per capita income of California is :", per_capital_income)

Per capita income of California is : $56,791


### As we have concluded that we can choose one location in California, we then check for the Per Capita Income of the state. As it is more than 50,000 USD, it can be considered as one of the preferable location to set up a New Office

### Let us now check which location in California would be the best to start a new marketing office

In [39]:
# Getting coordinates of California
lat_California = df['Latitude'].iloc[row]
long_California = df['Longitude'].iloc[row]
print(lat_California, long_California)

33.7363 -117.883


#### Getting all the venues of California using Four Square API within under considered radius and Limits

In [40]:
# Getting the venues of California using four square API 

def getNearbyVenues1(name, latitudes, longitudes, radius):
    
    LIMIT = 250       
        # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitudes, 
            longitudes, 
            radius, 
            LIMIT)
            
        # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
   # print(results)
    venues_list=[]
    venues_list.append([(name,lat,lng,v['venue']['name'],v['venue']['location']['lat'],v['venue']['location']['lng'],v['venue']['categories'][0]['name'])for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 'Latitude', 'Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude','Category']
    return(nearby_venues)
 
new_California_venues = getNearbyVenues1(name = 'California City', latitudes = lat_California ,longitudes = long_California, radius = 2500)
new_California_venues.head()

,City,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Category
0,California City,38.3539,-121.9728,El Toro Carniceria,33.745213,-117.886839,Mexican Restaurant
1,California City,38.3539,-121.9728,El Gallo Giro,33.727420,-117.885829,Mexican Restaurant
2,California City,38.3539,-121.9728,Taqueria Los Grandes,33.748622,-117.892983,Mexican Restaurant
3,California City,38.3539,-121.9728,Alebrije's Grill Taco Truck,33.742663,-117.876176,Food Truck
4,California City,38.3539,-121.9728,Crave,33.748031,-117.871284,American Restaurant


In [41]:
venues_in_new_California = new_California_venues.copy(deep = True)
print (venues_in_new_California.shape)
venues_in_new_California.Category.unique()

(100, 7)


array(['Mexican Restaurant', 'Food Truck', 'American Restaurant', 'Café',
       'Art Gallery', 'Clothing Store', 'Market', 'Bar', 'Grocery Store',
       'Restaurant', 'Coffee Shop', 'Filipino Restaurant',
       'Hot Dog Joint', 'Indie Movie Theater', 'Brewery',
       'Latin American Restaurant', 'Bakery', 'Dessert Shop', 'Diner',
       'Burrito Place', 'Video Game Store', 'Beer Garden', 'Burger Joint',
       'Seafood Restaurant', 'Neighborhood', 'Convenience Store',
       'Taco Place', 'Cocktail Bar', 'Gay Bar', 'Buffet',
       'Fast Food Restaurant', 'Pharmacy', 'Gym / Fitness Center',
       'Chinese Restaurant', 'Juice Bar', 'Sandwich Place', 'Wine Bar',
       'New American Restaurant', 'BBQ Joint', 'Concert Hall', 'Theater',
       'Department Store', 'Performing Arts Venue', 'Video Store',
       'Pizza Place', 'Rental Car Location', 'Antique Shop',
       'Discount Store', 'Supermarket', 'Mobile Phone Shop', 'Park',
       'Football Stadium'], dtype=object)

### Since we have got 100 categories, we are now giving weights for each category for better results

In [42]:
#copying the data frame in and giving weights for each category

k = new_California_venues.copy(deep = True)
new_weightage_dict= {'American Restaurant': 2,
'Spanish Restaurant': 2,
'Pizza Place': 2,
'Latin American Restaurant': 2,
'Restaurant': 2,
'Seafood Restaurant': 2,
'Fish Market': 2,
'Cuban Restaurant': 2,
'Soccer Field': 5,
'Museum': 3,
'Baseball Stadium': 2,
'Mexican Restaurant': 2,
'Wine Bar': 2,
'Train Station': 2,
'Sandwich Place': 2,
'Ice Cream Shop': 2,
'Salon / Barbershop': 2,
'Music Venue': 2,
'Frozen Yogurt Shop': 2,
'Smoke Shop': 2,
'Argentinian Restaurant': 2,
'Art Museum': 2,
'Shopping Mall': 5,
'Jewelry Store': 2,
'Tapas Restaurant': 2,
'Indie Movie Theater': 2,
'Greek Restaurant': 2,
'Electronics Store': 5,
'Italian Restaurant': 2,
'Café': 2,
'Bar': 2,
'Hotel': 2,
'Gastropub': 2,
'Coffee Shop': 2,
'Theater': 2,
'Grocery Store': 2,
'Brazilian Restaurant': 2,
'Gym': 2,
'Bagel Shop': 2,
'Thai Restaurant': 2,
'Beer Bar': 2,
'Cosmetics Shop': 2,
'Art Gallery': 2,
'Bakery': 2,
'Indonesian Restaurant': 2,
'Peruvian Restaurant': 2,
'Japanese Restaurant': 2,
'Nightclub': 2,
'Spa': 2,
'Tex-Mex Restaurant': 2,
'Residential Building (Apartment / Condo)': 2,
'Pet Service': 2,
'Cocktail Bar': 2,
'Basketball Stadium': 2,
'Asian Restaurant': 2,
'Event Space': 5,
'Chinese Restaurant': 2,
'Mediterranean Restaurant': 2,
'Steakhouse': 2,
'Park': 2, 
}


#### Plotting all the venues that we have got from the Four Square API

In [43]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [44]:
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

# create map of the venues that we have using latitude and longitudes
venues_map = folium.Map(location=[lat_California, long_California],tiles='Stamen Toner', zoom_start=15) # generate map centred around Florida city

# add California City as a green circle mark
folium.features.CircleMarker(
    [lat_California, long_California],
    radius=20,
    popup='California city',
    fill=True,
    color='green',
    fill_color='green',
    fill_opacity=0.6
    ).add_to(venues_map)

In [45]:
# add all the venuew of the California  city to the map as blue circle markers
for lat, lng, label in zip(venues_in_new_California['Venue Latitude'], venues_in_new_California['Venue Longitude'], venues_in_new_California['Venue']):
    label=folium.Popup(label,parse_html=True)
    folium.features.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        parse_html = False).add_to(venues_map)
venues_map

#### Assigning weights to each category, same as we gave for each city 

In [46]:
# Calculating new weights for our data frame as we have given weights for all categories

allVenuesinCity1 = k['Category']

f_weights1 = []
for i in allVenuesinCity1:
    if i in new_weightage_dict.keys():
        f_weights1.append(new_weightage_dict[i])
    else :
        f_weights1.append(0)
k['weights'] = f_weights1;
k.head()

,City,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Category,weights
0,California City,38.3539,-121.9728,El Toro Carniceria,33.745213,-117.886839,Mexican Restaurant,2
1,California City,38.3539,-121.9728,El Gallo Giro,33.727420,-117.885829,Mexican Restaurant,2
2,California City,38.3539,-121.9728,Taqueria Los Grandes,33.748622,-117.892983,Mexican Restaurant,2
3,California City,38.3539,-121.9728,Alebrije's Grill Taco Truck,33.742663,-117.876176,Food Truck,0
4,California City,38.3539,-121.9728,Crave,33.748031,-117.871284,American Restaurant,2


In [47]:
# Dropping unnecessary columns 

newframe = k[['City','Category','weights']].copy()
newframe = k.groupby(['Category']).mean()
newframe.drop(columns = ["Latitude", "Longitude"], inplace = True)
newframe 

,Venue Latitude,Venue Longitude,weights
Category,,,
American Restaurant,33.739037,-117.872515,2
Antique Shop,33.717790,-117.868286,0
Art Gallery,33.746809,-117.869412,2
BBQ Joint,33.747264,-117.886817,0
Bakery,33.739144,-117.876187,2
Bar,33.747376,-117.868451,2
Beer Garden,33.748151,-117.865371,0
Brewery,33.748111,-117.870060,0
Buffet,33.715060,-117.884451,0


## Using K Means algorithm to cluster the venues and calculating the weights for each cluster to decide which cluster would be the best area to have a new marketing office.

In [48]:
# Cluster them using K means algorithm 
from scipy import stats
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
#Standardize
clmns = ['weights','Venue Latitude', 'Venue Longitude']
df_tr_std = stats.zscore(newframe[clmns])
#Cluster the data
kmeans = KMeans(n_clusters=3, random_state=0).fit(df_tr_std)
labels = kmeans.labels_
newframe['clusters'] = labels
#Add the column into our list
clmns.extend(['clusters'])
#Lets analyze the clusters
kframe = newframe[clmns].groupby(['Category']).mean()
kframe = kframe.reset_index(drop = False)
kframe.head()

,Category,weights,Venue Latitude,Venue Longitude,clusters
0,American Restaurant,2,33.739037,-117.872515,2
1,Antique Shop,0,33.717790,-117.868286,1
2,Art Gallery,2,33.746809,-117.869412,2
3,BBQ Joint,0,33.747264,-117.886817,1
4,Bakery,2,33.739144,-117.876187,2


In [49]:
#new group by clusters and add weights of each cluster 
finalWeight = kframe.groupby(['clusters']).mean()
finalWeight

,weights,Venue Latitude,Venue Longitude
clusters,,,
0,0.923077,33.726304,-117.886059
1,0.000000,33.742656,-117.871449
2,2.000000,33.745292,-117.869698


In [50]:
# Final coordinates of the place where we will be setting up a new marketing office is the one that has maximum weight for, in the above data frame
lat1 = 33.745292
long1 = -117.869698

#### As we have a location with the maximum weight, let us plot the same in the graph with a circle of radius 50M such that CEO can have a new office.

In [51]:
# create map of the venues that we have using latitude and longitudes
final_map = folium.Map(location=[lat1, long1],tiles='Stamen Toner', zoom_start=15) # generate map centred around California

# add prefered location in the City as a green circle mark
folium.features.CircleMarker(
    [lat1, long1],
    radius=50,
    popup='New Office can be set up within this circle',
    fill=True,
    color='yellow',
    fill_color='yellow',
    fill_opacity=0.6
    ).add_to(final_map)
final_map

### So, we finally got a better place in the California